# HyperCLOVAX 모델 Batch Size 비교 분석

이 노트북은 원본 모델과 서로 다른 Batch Size로 미세조정된 HyperCLOVAX 모델의 성능을 비교합니다.

## 분석 목표
- 네 모델 간 정량적 성능 비교 (BLEU, ROUGE, 문자 정확도)
- 정성적 분석 (실제 출력 예시 비교)
- 추론 시간 및 효율성 비교
- Batch Size 미세조정 효과 분석
- 결과 시각화

## 모델 정보
- **원본 모델**: `naver-hyperclovax/HyperCLOVAX-SEED-Text-Instruct-0.5B`
- **Batch Size 1 모델**: `hyperclova-deobfuscation-lora-1-batch-size` (가정된 경로)
- **Batch Size 2 모델**: `hyperclova-deobfuscation-lora-2-batch-size` (가정된 경로)
- **Batch Size 4 모델**: `hyperclova-deobfuscation-lora-4-batch-size` (가정된 경로)
- **테스트 데이터**: `testdata.csv` (1,002 샘플)

## 1. 환경 설정 및 라이브러리 설치

In [ ]:
# GPU 확인
!nvidia-smi

# 필수 패키지 설치
!pip install -q transformers
!pip install -q peft
!pip install -q torch
!pip install -q datasets
!pip install -q evaluate
!pip install -q rouge-score
!pip install -q sacrebleu
!pip install -q sentencepiece
!pip install -q protobuf
!pip install -q matplotlib
!pip install -q seaborn
!pip install -q plotly
!pip install -q pandas
!pip install -q numpy
!pip install -q scikit-learn
!pip install -q tqdm
!pip install -q python-Levenshtein

print("패키지 설치 완료")

In [ ]:
import torch
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel
from evaluate import load
from tqdm import tqdm
import time
import warnings
import os
warnings.filterwarnings('ignore')

# matplotlib 및 seaborn 설정
# Colab 환경에서 한글 폰트 설정 (필요시)
# !sudo apt-get install -y fonts-nanum
# !sudo fc-cache -fv
# !rm ~/.cache/matplotlib -rf
# plt.rc('font', family='NanumBarunGothic') 
plt.rcParams['font.family'] = 'DejaVu Sans' # 기본 폰트 사용
sns.set_style("whitegrid")
sns.set_palette("husl")

# 장치 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"사용 중인 장치: {device}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"GPU 메모리: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB")

## 2. 데이터 로딩 및 전처리

In [ ]:
# Google Drive 연결 (Colab에서 실행 시)
try:
    from google.colab import drive
    import shutil
    
    # 기존 마운트 포인트가 있으면 정리
    mount_point = '/content/drive'
    if os.path.exists(mount_point):
        try:
            # 마운트 해제 시도
            print("기존 마운트 포인트 정리 중...")
            os.system(f'fusermount -u {mount_point} 2>/dev/null || true')
            shutil.rmtree(mount_point, ignore_errors=True)
        except:
            pass
    
    # Google Drive 마운트
    drive.mount(mount_point, force_remount=True)
    
    # 경로 설정 (Google Drive 기준)
    BASE_PATH = '/content/drive/MyDrive/' # 사용자의 Drive 경로에 맞게 수정 필요
    MODEL_BS1_PATH = BASE_PATH + 'hyperclova-deobfuscation-lora-1-batch-size' # 실제 경로로 수정 필요
    MODEL_BS2_PATH = BASE_PATH + 'hyperclova-deobfuscation-lora-2-batch-size' # 실제 경로로 수정 필요
    MODEL_BS4_PATH = BASE_PATH + 'hyperclova-deobfuscation-lora-4-batch-size' # 실제 경로로 수정 필요
    TEST_DATA_PATH = BASE_PATH + 'testdata.csv' # 실제 경로로 수정 필요
    
    # Google Drive 루트에 전용 분석 결과 폴더 생성
    analysis_root_dir = os.path.join(BASE_PATH, 'HyperCLOVAX_BatchSize_Analysis_Results')
    os.makedirs(analysis_root_dir, exist_ok=True)
    print(f"분석 결과 루트 폴더 생성: {analysis_root_dir}")
    
except ImportError:
    # 로컬 실행 시 (경로를 로컬 환경에 맞게 수정)
    BASE_PATH = './'
    MODEL_BS1_PATH = './hyperclova-deobfuscation-lora-1-batch-size' # 실제 경로로 수정 필요
    MODEL_BS2_PATH = './hyperclova-deobfuscation-lora-2-batch-size' # 실제 경로로 수정 필요
    MODEL_BS4_PATH = './hyperclova-deobfuscation-lora-4-batch-size' # 실제 경로로 수정 필요
    TEST_DATA_PATH = './testdata.csv' # 실제 경로로 수정 필요
    
    # 로컬용 분석 결과 폴더
    analysis_root_dir = './HyperCLOVAX_BatchSize_Analysis_Results'
    os.makedirs(analysis_root_dir, exist_ok=True)
    print(f"분석 결과 루트 폴더 생성: {analysis_root_dir}")

# 이미지 저장 폴더 생성 (분석 결과 폴더 내에)
image_save_dir = os.path.join(analysis_root_dir, 'visualization_images')
os.makedirs(image_save_dir, exist_ok=True)
print(f"이미지 저장 폴더 생성: {image_save_dir}")

print(f"\n경로 설정 완료:")
print(f"Batch Size 1 모델 경로: {MODEL_BS1_PATH}")
print(f"Batch Size 2 모델 경로: {MODEL_BS2_PATH}")
print(f"Batch Size 4 모델 경로: {MODEL_BS4_PATH}")
print(f"테스트 데이터 경로: {TEST_DATA_PATH}")
print(f"분석 결과 저장 경로: {analysis_root_dir}")
print(f"이미지 저장 경로: {image_save_dir}")

In [ ]:
# 테스트 데이터 로드
try:
    test_df = pd.read_csv(TEST_DATA_PATH)
    print(f"테스트 데이터 크기: {len(test_df)} 샘플")
    print(f"컬럼 목록: {test_df.columns.tolist()}")
    print("\n첫 5개 샘플:")
    print(test_df.head())
    
    # 데이터 통계
    print("\n데이터 통계:")
    print(f"- 총 샘플 수: {len(test_df)}")
    print(f"- 원본 텍스트 평균 길이: {test_df['original'].str.len().mean():.1f}")
    print(f"- 난독화 텍스트 평균 길이: {test_df['obfuscated'].str.len().mean():.1f}")
except FileNotFoundError:
    print(f"오류: 테스트 데이터 파일({TEST_DATA_PATH})을 찾을 수 없습니다. 경로를 확인해주세요.")
    # 필요한 경우 여기서 실행 중단 또는 기본 데이터프레임 생성
    test_df = pd.DataFrame() # 빈 데이터프레임으로 초기화하여 이후 코드 오류 방지

## 3. 모델 로딩

In [ ]:
# 베이스 모델 이름 설정
BASE_MODEL_NAME = "naver-hyperclovax/HyperCLOVAX-SEED-Text-Instruct-0.5B"

# 토크나이저 로드 (하나의 모델 경로에서 로드)
print("토크나이저 로딩 중...")
try:
    # Batch Size 1 모델 경로에서 토크나이저 로드 시도
    tokenizer = AutoTokenizer.from_pretrained(MODEL_BS1_PATH)
    print(f"토크나이저 로딩 완료 (어휘 크기: {len(tokenizer)})")
except Exception as e:
    print(f"오류: {MODEL_BS1_PATH}에서 토크나이저 로딩 실패: {e}")
    print(f"베이스 모델({BASE_MODEL_NAME})에서 토크나이저 로딩 시도...")
    try:
        tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL_NAME)
        print(f"베이스 모델 토크나이저 로딩 완료 (어휘 크기: {len(tokenizer)})")
    except Exception as e2:
        print(f"오류: 베이스 모델({BASE_MODEL_NAME})에서도 토크나이저 로딩 실패: {e2}")
        # 토크나이저 로딩 실패 시 이후 코드 실행 불가
        tokenizer = None

In [ ]:
def load_lora_model(model_path, model_name):
    """LoRA 모델을 로드합니다"""
    print(f"\n{model_name} ({model_path}) 로딩 중...")
    try:
        # 베이스 모델 로드
        base_model = AutoModelForCausalLM.from_pretrained(
            BASE_MODEL_NAME,
            torch_dtype=torch.float16,
            device_map="auto" # GPU 자동 할당
        )
        print(f" - 베이스 모델({BASE_MODEL_NAME}) 로드 완료")
        
        # LoRA 어댑터 적용
        model = PeftModel.from_pretrained(base_model, model_path)
        model = model.to(device) # 모델을 지정된 장치로 이동
        model.eval() # 평가 모드로 설정
        print(f" - LoRA 어댑터 적용 완료")
        print(f"{model_name} 로딩 완료")
        return model
    except Exception as e:
        print(f"오류: {model_name} 로딩 실패: {e}")
        print(f" - 경로({model_path}) 또는 모델 파일 확인 필요")
        return None

def load_base_model():
    """원본 베이스 모델을 로드합니다"""
    print(f"\n원본 모델 ({BASE_MODEL_NAME}) 로딩 중...")
    try:
        base_model = AutoModelForCausalLM.from_pretrained(
            BASE_MODEL_NAME,
            torch_dtype=torch.float16,
            device_map="auto"
        )
        base_model = base_model.to(device)
        base_model.eval() # 평가 모드로 설정
        print(f"원본 모델 로딩 완료")
        return base_model
    except Exception as e:
        print(f"오류: 원본 모델 로딩 실패: {e}")
        return None

# 모델 로드
models = {}
models['원본 모델'] = load_base_model()
models['Batch Size 1 모델'] = load_lora_model(MODEL_BS1_PATH, "Batch Size 1 모델")
models['Batch Size 2 모델'] = load_lora_model(MODEL_BS2_PATH, "Batch Size 2 모델")
models['Batch Size 4 모델'] = load_lora_model(MODEL_BS4_PATH, "Batch Size 4 모델")

# 로드 성공한 모델만 필터링
loaded_models = {name: model for name, model in models.items() if model is not None}

if len(loaded_models) > 0:
    print(f"\n총 {len(loaded_models)}개 모델 로딩 완료: {list(loaded_models.keys())}")
else:
    print("\n오류: 로드된 모델이 없습니다. 경로 및 파일 확인 후 다시 시도해주세요.")

## 4. 추론 함수 정의

In [ ]:
def generate_deobfuscated_text(model, tokenizer, obfuscated_text, max_length=256):
    """난독화된 텍스트를 입력받아 원본 텍스트 생성"""
    if tokenizer is None:
        print("오류: 토크나이저가 로드되지 않아 추론을 진행할 수 없습니다.")
        return "토크나이저 로딩 오류", 0.0
        
    prompt = f"""### 지시사항:
다음 난독화된 한국어 텍스트를 원래 텍스트로 복원해주세요.

난독화된 텍스트: {obfuscated_text}

### 응답:
"""
    
    try:
        inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=512)
        inputs = {k: v.to(device) for k, v in inputs.items()}
        
        start_time = time.time()
        
        with torch.no_grad():
            outputs = model.generate(
                **inputs,
                max_new_tokens=max_length,
                do_sample=True, # 샘플링 사용
                temperature=0.7, # 약간의 다양성 부여
                top_p=0.9, # 상위 90% 확률 누적 토큰만 고려
                pad_token_id=tokenizer.eos_token_id, # 패딩 토큰 설정
                eos_token_id=tokenizer.eos_token_id # 종료 토큰 설정
            )
        
        inference_time = time.time() - start_time
        
        # 생성된 토큰 ID만 디코딩 (입력 프롬프트 제외)
        output_tokens = outputs[0][inputs['input_ids'].shape[1]:]
        response = tokenizer.decode(output_tokens, skip_special_tokens=True)
        
        # 후처리: 불필요한 공백 제거 및 종료 토큰 관련 문자열 제거
        response = response.strip()
        # 가끔 <|endoftext|> 같은 특수 토큰이 포함될 경우 제거
        response = response.replace('<|endoftext|>', '').strip()
            
        return response, inference_time
        
    except Exception as e:
        print(f"오류: 추론 중 오류 발생 - {e}")
        return "추론 오류", 0.0

print("추론 함수 정의 완료")

## 5. 성능 평가 메트릭 정의

In [ ]:
# 평가 메트릭 로드
try:
    bleu = load("bleu")
    rouge = load("rouge")
    print("BLEU 및 ROUGE 메트릭 로드 완료")
except Exception as e:
    print(f"오류: 평가 메트릭 로드 실패 - {e}")
    bleu = None
    rouge = None

def calculate_character_accuracy(pred, ref):
    """문자 단위 정확도 계산 (Levenshtein 거리 기반 유사도)
    pip install python-Levenshtein 필요
    """
    try:
        import Levenshtein
        if len(ref) == 0:
            return 1.0 if len(pred) == 0 else 0.0
        distance = Levenshtein.distance(pred, ref)
        # 유사도 = 1 - (편집 거리 / 더 긴 문자열 길이)
        similarity = 1.0 - (distance / max(len(pred), len(ref)))
        return max(0.0, similarity) # 0 미만 값 방지
    except ImportError:
        print("경고: python-Levenshtein 패키지가 설치되지 않았습니다. 문자 정확도 계산을 건너<0xEB><0x9A><0x8D>니다.")
        print("설치 방법: !pip install python-Levenshtein")
        # 대체 계산법 (간단한 일치율)
        if len(ref) == 0:
             return 1.0 if len(pred) == 0 else 0.0
        matches = sum(1 for i, char in enumerate(pred) if i < len(ref) and char == ref[i])
        return matches / len(ref)
    except Exception as e:
        print(f"문자 정확도 계산 중 오류: {e}")
        return 0.0

def calculate_exact_match(pred, ref):
    """완전 일치 여부 (공백 제거 후 비교)"""
    return 1.0 if pred.strip() == ref.strip() else 0.0

def calculate_metrics(predictions, references):
    """모든 메트릭 계산"""
    metrics_results = {
        'bleu': 0.0,
        'rouge1': 0.0,
        'rouge2': 0.0,
        'rougeL': 0.0,
        'char_accuracy': 0.0,
        'exact_match': 0.0,
        'char_accuracies': [],
        'exact_matches': []
    }
    
    if not predictions or not references:
        print("경고: 예측 또는 참조 목록이 비어있어 메트릭 계산을 건너<0xEB><0x9A><0x8D>니다.")
        return metrics_results
        
    # BLEU 계산
    if bleu:
        try:
            # references를 sacrebleu 형식에 맞게 변환: [[ref1], [ref2], ...] -> [[ref1_sent1, ref1_sent2,...], [ref2_sent1,...]]
            # 여기서는 각 샘플당 참조가 하나이므로 [[ref] for ref in references] 사용
            bleu_score = bleu.compute(predictions=predictions, references=[[ref] for ref in references])
            metrics_results['bleu'] = bleu_score['bleu'] if bleu_score and 'bleu' in bleu_score else 0.0
        except Exception as e:
            print(f"BLEU 계산 중 오류: {e}")
            metrics_results['bleu'] = 0.0
    else:
        print("BLEU 메트릭이 로드되지 않아 계산을 건너<0xEB><0x9A><0x8D>니다.")
    
    # ROUGE 계산
    if rouge:
        try:
            rouge_scores = rouge.compute(predictions=predictions, references=references)
            metrics_results['rouge1'] = rouge_scores['rouge1'] if rouge_scores and 'rouge1' in rouge_scores else 0.0
            metrics_results['rouge2'] = rouge_scores['rouge2'] if rouge_scores and 'rouge2' in rouge_scores else 0.0
            metrics_results['rougeL'] = rouge_scores['rougeL'] if rouge_scores and 'rougeL' in rouge_scores else 0.0
        except Exception as e:
            print(f"ROUGE 계산 중 오류: {e}")
            metrics_results['rouge1'] = 0.0
            metrics_results['rouge2'] = 0.0
            metrics_results['rougeL'] = 0.0
    else:
        print("ROUGE 메트릭이 로드되지 않아 계산을 건너<0xEB><0x9A><0x8D>니다.")
    
    # 문자 정확도 및 완전 일치율 계산
    char_accuracies = []
    exact_matches = []
    for pred, ref in zip(predictions, references):
        char_accuracies.append(calculate_character_accuracy(pred, ref))
        exact_matches.append(calculate_exact_match(pred, ref))
        
    if char_accuracies:
        metrics_results['char_accuracy'] = np.mean(char_accuracies)
        metrics_results['char_accuracies'] = char_accuracies
    if exact_matches:
        metrics_results['exact_match'] = np.mean(exact_matches)
        metrics_results['exact_matches'] = exact_matches
        
    return metrics_results

print("평가 메트릭 정의 완료")

## 6. 모델 성능 평가 실행

In [ ]:
def evaluate_model(model, model_name, tokenizer, test_df, sample_size=None):
    """모델 성능 평가"""
    if model is None:
        print(f"경고: {model_name}이(가) 로드되지 않아 평가를 건너<0xEB><0x9A><0x8D>니다.")
        return None
    if tokenizer is None:
        print(f"경고: 토크나이저가 로드되지 않아 {model_name} 평가를 건너<0xEB><0x9A><0x8D>니다.")
        return None
    if test_df.empty:
        print(f"경고: 테스트 데이터가 비어있어 {model_name} 평가를 건너<0xEB><0x9A><0x8D>니다.")
        return None
        
    if sample_size and sample_size < len(test_df):
        test_data = test_df.sample(n=sample_size, random_state=42).reset_index(drop=True)
        print(f"\n{model_name} 평가 시작 ({sample_size}개 샘플 무작위 추출)")
    else:
        test_data = test_df.copy()
        print(f"\n{model_name} 평가 시작 ({len(test_data)}개 전체 샘플)")
    
    predictions = []
    inference_times = []
    references = test_data['original'].tolist() # 참조 텍스트 미리 준비
    
    # tqdm 설정: desc에 모델 이름 포함, unit='샘플'
    progress_bar = tqdm(test_data.iterrows(), total=len(test_data), desc=f"{model_name} 평가", unit='샘플')
    
    for idx, row in progress_bar:
        obfuscated = row['obfuscated']
        # generate_deobfuscated_text 함수에 tokenizer 전달
        pred, inf_time = generate_deobfuscated_text(model, tokenizer, obfuscated)
        predictions.append(pred)
        inference_times.append(inf_time)
        # 진행률 표시줄에 현재 처리 중인 인덱스 표시 (선택 사항)
        # progress_bar.set_postfix({'idx': idx}) 
    
    # 메트릭 계산
    metrics = calculate_metrics(predictions, references)
    
    # 추론 시간 통계 (0인 경우 제외)
    valid_inference_times = [t for t in inference_times if t > 0]
    avg_inference_time = np.mean(valid_inference_times) if valid_inference_times else 0.0
    total_inference_time = np.sum(inference_times)
    
    results = {
        'model_name': model_name,
        'predictions': predictions,
        'references': references,
        'inference_times': inference_times,
        'avg_inference_time': avg_inference_time,
        'total_inference_time': total_inference_time,
        'test_data_indices': test_data.index.tolist(), # 평가에 사용된 데이터 인덱스 저장
        **metrics # 계산된 메트릭 결과 통합
    }
    
    print(f"{model_name} 평가 완료")
    print(f" - 평균 추론 시간: {avg_inference_time:.3f}초 (유효 샘플 기준)")
    print(f" - 총 추론 시간: {total_inference_time:.1f}초")
    print(f" - BLEU: {metrics['bleu']:.4f}, ROUGE-L: {metrics['rougeL']:.4f}, Char Acc: {metrics['char_accuracy']:.4f}, Exact Match: {metrics['exact_match']:.4f}")
    
    return results

# --- 평가 실행 --- #
SAMPLE_SIZE = 200  # None으로 설정하면 전체 데이터셋 사용 (시간이 오래 걸릴 수 있음)
evaluation_results = {}

if not loaded_models:
    print("오류: 평가할 모델이 로드되지 않았습니다.")
elif test_df.empty:
    print("오류: 테스트 데이터가 비어있어 평가를 진행할 수 없습니다.")
elif tokenizer is None:
    print("오류: 토크나이저가 로드되지 않아 평가를 진행할 수 없습니다.")
else:
    print(f"모델 성능 평가를 시작합니다 (샘플 크기: {SAMPLE_SIZE if SAMPLE_SIZE else '전체'})...")
    
    # 로드된 모델들에 대해 순차적으로 평가 실행
    for model_name, model_instance in loaded_models.items():
        evaluation_results[model_name] = evaluate_model(model_instance, model_name, tokenizer, test_df, SAMPLE_SIZE)
        # 메모리 관리: 평가 끝난 모델의 GPU 메모리 해제 시도 (선택 사항)
        # del model_instance
        # if torch.cuda.is_available():
        #     torch.cuda.empty_cache()
            
    # 평가가 완료된 결과만 필터링
    completed_evaluations = {name: res for name, res in evaluation_results.items() if res is not None}
    
    if completed_evaluations:
        print(f"\n총 {len(completed_evaluations)}개 모델 평가 완료!")
    else:
        print("\n오류: 성공적으로 평가된 모델이 없습니다.")

## 7. 성능 비교 결과 출력

In [ ]:
# 성능 비교 표 생성
if 'completed_evaluations' in locals() and completed_evaluations:
    metrics_to_display = ['bleu', 'rouge1', 'rouge2', 'rougeL', 'char_accuracy', 'exact_match', 'avg_inference_time']
    metric_names_kr = ['BLEU 점수', 'ROUGE-1', 'ROUGE-2', 'ROUGE-L', '문자 정확도', '정확 일치율', '평균 추론 시간(초)']
    
    comparison_data = {'메트릭': metric_names_kr}
    
    # 평가된 모델 순서대로 데이터 추가 (원본 모델 우선)
    model_order = ['원본 모델'] + [name for name in completed_evaluations if name != '원본 모델']
    
    for model_name in model_order:
        if model_name in completed_evaluations:
            results = completed_evaluations[model_name]
            model_column = []
            for metric_key in metrics_to_display:
                value = results.get(metric_key, 0.0) # 메트릭 값이 없는 경우 0.0으로 처리
                if metric_key == 'avg_inference_time':
                    model_column.append(f"{value:.3f}") # 소수점 3자리
                else:
                    model_column.append(f"{value:.4f}") # 소수점 4자리
            comparison_data[model_name] = model_column
            
    comparison_df = pd.DataFrame(comparison_data)
    
    print("--- 모델 성능 비교표 ---")
    print(comparison_df.to_string(index=False))
    
    # 결과를 CSV 파일로 저장
    comparison_csv_path = os.path.join(analysis_root_dir, 'model_performance_comparison_batchsize.csv')
    try:
        comparison_df.to_csv(comparison_csv_path, index=False, encoding='utf-8-sig')
        print(f"\n성능 비교표를 CSV 파일로 저장했습니다: {comparison_csv_path}")
    except Exception as e:
        print(f"오류: 성능 비교표 CSV 저장 실패 - {e}")

else:
    print("오류: 평가 결과가 없어 성능 비교표를 생성할 수 없습니다.")

## 8. 결과 시각화

In [ ]:
# 시각화 함수 정의
def plot_comparison_bar_chart(df, title, filename):
    """성능 비교 막대 그래프 생성"""
    if df.empty:
        print(f"경고: 데이터프레임이 비어있어 '{title}' 그래프를 생성할 수 없습니다.")
        return
        
    try:
        # '메트릭' 컬럼을 인덱스로 설정
        df_plot = df.set_index('메트릭')
        # 숫자형으로 변환 시도, 오류 발생 시 0으로 대체
        df_plot = df_plot.apply(pd.to_numeric, errors='coerce').fillna(0.0)
        
        # 추론 시간 제외하고 시각화 (스케일 차이 때문)
        metrics_for_plot = [m for m in df_plot.index if '시간' not in m]
        df_plot_filtered = df_plot.loc[metrics_for_plot]
        
        if df_plot_filtered.empty:
             print(f"경고: '{title}' 그래프에 표시할 유효한 메트릭 데이터가 없습니다.")
             return
             
        ax = df_plot_filtered.plot(kind='bar', figsize=(14, 8), rot=0)
        plt.title(title, fontsize=16)
        plt.ylabel('점수', fontsize=12)
        plt.xlabel('평가 메트릭', fontsize=12)
        plt.xticks(fontsize=10)
        plt.yticks(fontsize=10)
        plt.legend(title='모델', fontsize=10, title_fontsize=11)
        plt.tight_layout()
        
        # 그래프 파일 저장
        filepath = os.path.join(image_save_dir, filename)
        plt.savefig(filepath)
        print(f"그래프 저장 완료: {filepath}")
        plt.show()
        plt.close() # 그래프 창 닫기
        
    except Exception as e:
        print(f"오류: '{title}' 그래프 생성 중 오류 발생 - {e}")

# 메인 성능 지표 비교 그래프
if 'comparison_df' in locals() and not comparison_df.empty:
    plot_comparison_bar_chart(comparison_df, "모델별 주요 성능 지표 비교 (Batch Size)", "batchsize_main_metrics_comparison.png")
else:
    print("성능 비교 데이터가 없어 메인 성능 지표 그래프를 생성할 수 없습니다.")

In [ ]:
# 추론 시간 비교 그래프
def plot_inference_time_comparison(df, title, filename):
    """추론 시간 비교 막대 그래프 생성"""
    if df.empty:
        print(f"경고: 데이터프레임이 비어있어 '{title}' 그래프를 생성할 수 없습니다.")
        return
        
    try:
        # '평균 추론 시간(초)' 행만 선택
        inference_time_data = df[df['메트릭'] == '평균 추론 시간(초)'].set_index('메트릭')
        # 숫자형으로 변환 시도, 오류 발생 시 0으로 대체
        inference_time_data = inference_time_data.apply(pd.to_numeric, errors='coerce').fillna(0.0)
        
        if inference_time_data.empty:
             print(f"경고: '{title}' 그래프에 표시할 추론 시간 데이터가 없습니다.")
             return
             
        ax = inference_time_data.T.plot(kind='bar', figsize=(10, 6), legend=False, rot=0)
        plt.title(title, fontsize=16)
        plt.ylabel('평균 추론 시간 (초)', fontsize=12)
        plt.xlabel('모델', fontsize=12)
        plt.xticks(fontsize=10)
        plt.yticks(fontsize=10)
        plt.tight_layout()
        
        # 막대 위에 값 표시
        for container in ax.containers:
            ax.bar_label(container, fmt='%.3f')
            
        # 그래프 파일 저장
        filepath = os.path.join(image_save_dir, filename)
        plt.savefig(filepath)
        print(f"그래프 저장 완료: {filepath}")
        plt.show()
        plt.close()
        
    except Exception as e:
        print(f"오류: '{title}' 그래프 생성 중 오류 발생 - {e}")

# 추론 시간 비교 그래프 실행
if 'comparison_df' in locals() and not comparison_df.empty:
    plot_inference_time_comparison(comparison_df, "모델별 평균 추론 시간 비교 (Batch Size)", "batchsize_inference_time_comparison.png")
else:
    print("성능 비교 데이터가 없어 추론 시간 비교 그래프를 생성할 수 없습니다.")

In [ ]:
# 개별 메트릭 분포 시각화 (Box Plot 등)
def plot_metric_distribution(results_dict, metric_key, metric_name_kr, title, filename):
    """개별 메트릭 분포 시각화 (Box Plot)"""
    if not results_dict:
        print(f"경고: 평가 결과가 없어 '{title}' 그래프를 생성할 수 없습니다.")
        return
        
    plot_data = []
    model_names = []
    
    # 모델 순서 정의 (원본 모델 우선)
    model_order = ['원본 모델'] + [name for name in results_dict if name != '원본 모델']
    
    for model_name in model_order:
        if model_name in results_dict:
            results = results_dict[model_name]
            # 메트릭 키 이름 변경 (예: 'char_accuracies' -> 'char_accuracy')
            individual_metric_key = metric_key + 's' if metric_key in ['char_accuracy', 'exact_match'] else metric_key
            if individual_metric_key in results and isinstance(results[individual_metric_key], list):
                plot_data.append(results[individual_metric_key])
                model_names.append(model_name)
            elif metric_key in results: # 단일 값 메트릭 (예: bleu, rouge) - 분포 시각화 부적합
                print(f"정보: '{metric_name_kr}'은(는) 단일 값 메트릭이므로 분포 시각화 대신 평균값을 사용합니다.")
                # 단일 값이라도 표시하려면 다른 방식 필요 (예: 점 그래프)
            else:
                 print(f"경고: {model_name} 결과에 '{individual_metric_key}' 또는 '{metric_key}' 데이터가 없습니다.")
                 
    if not plot_data:
        print(f"경고: '{title}' 그래프에 표시할 데이터가 없습니다.")
        return
        
    try:
        plt.figure(figsize=(12, 7))
        sns.boxplot(data=plot_data)
        plt.xticks(ticks=range(len(model_names)), labels=model_names, rotation=10, ha='right')
        plt.title(title, fontsize=16)
        plt.ylabel(metric_name_kr, fontsize=12)
        plt.xlabel('모델', fontsize=12)
        plt.tight_layout()
        
        # 그래프 파일 저장
        filepath = os.path.join(image_save_dir, filename)
        plt.savefig(filepath)
        print(f"그래프 저장 완료: {filepath}")
        plt.show()
        plt.close()
        
    except Exception as e:
        print(f"오류: '{title}' 그래프 생성 중 오류 발생 - {e}")

# 문자 정확도 분포 시각화
if 'completed_evaluations' in locals() and completed_evaluations:
    plot_metric_distribution(completed_evaluations, 'char_accuracy', '문자 정확도', "모델별 문자 정확도 분포 (Batch Size)", "batchsize_char_accuracy_distribution.png")
else:
    print("평가 결과가 없어 문자 정확도 분포 그래프를 생성할 수 없습니다.")

# 정확 일치율 분포 시각화 (Box plot보다는 Bar plot이 적합할 수 있음)
# 여기서는 Box Plot으로 시도
if 'completed_evaluations' in locals() and completed_evaluations:
    plot_metric_distribution(completed_evaluations, 'exact_match', '정확 일치율', "모델별 정확 일치율 분포 (Batch Size)", "batchsize_exact_match_distribution.png")
else:
    print("평가 결과가 없어 정확 일치율 분포 그래프를 생성할 수 없습니다.")

## 9. 정성적 분석 (출력 예시 비교)

In [ ]:
# 정성적 비교를 위한 데이터프레임 생성
def create_qualitative_comparison_df(results_dict, num_samples=10):
    """정성적 비교를 위한 샘플 데이터프레임 생성"""
    if not results_dict:
        print("경고: 평가 결과가 없어 정성적 비교 데이터프레임을 생성할 수 없습니다.")
        return pd.DataFrame()
        
    # 기준 모델 (예: 원본 모델)의 결과 가져오기
    base_model_name = '원본 모델'
    if base_model_name not in results_dict:
        # 원본 모델 없으면 첫 번째 모델 사용
        base_model_name = list(results_dict.keys())[0]
        
    base_results = results_dict[base_model_name]
    if 'test_data_indices' not in base_results or 'references' not in base_results or 'predictions' not in base_results:
        print(f"경고: {base_model_name} 결과에 필요한 데이터(인덱스, 참조, 예측)가 부족합니다.")
        return pd.DataFrame()
        
    # 평가에 사용된 원본 데이터에서 샘플 인덱스 가져오기
    eval_indices = base_results['test_data_indices']
    if not eval_indices:
        print("경고: 평가에 사용된 데이터 인덱스가 없습니다.")
        return pd.DataFrame()
        
    # 원본 test_df에서 해당 인덱스의 데이터 추출
    if 'test_df' not in globals() or test_df.empty:
        print("경고: 원본 test_df가 로드되지 않았거나 비어있습니다.")
        return pd.DataFrame()
        
    sampled_indices = np.random.choice(eval_indices, size=min(num_samples, len(eval_indices)), replace=False)
    qualitative_df = test_df.loc[sampled_indices, ['original', 'obfuscated']].copy()
    qualitative_df.rename(columns={'original': '정답 (Original)', 'obfuscated': '입력 (Obfuscated)'}, inplace=True)
    
    # 모델 순서 정의 (원본 모델 우선)
    model_order = ['원본 모델'] + [name for name in results_dict if name != '원본 모델']
    
    # 각 모델의 예측 결과 추가
    for model_name in model_order:
        if model_name in results_dict:
            results = results_dict[model_name]
            if 'predictions' in results and 'test_data_indices' in results:
                # 결과 딕셔너리 생성 (인덱스 -> 예측)
                pred_dict = dict(zip(results['test_data_indices'], results['predictions']))
                # 샘플된 인덱스에 해당하는 예측만 추출
                qualitative_df[f'예측 ({model_name})'] = [pred_dict.get(idx, 'N/A') for idx in sampled_indices]
            else:
                qualitative_df[f'예측 ({model_name})'] = '결과 없음'
                
    return qualitative_df

# 정성적 비교 실행 (15개 샘플)
if 'completed_evaluations' in locals() and completed_evaluations:
    qualitative_comparison_df = create_qualitative_comparison_df(completed_evaluations, num_samples=15)
    if not qualitative_comparison_df.empty:
        print("\n--- 정성적 비교 샘플 (15개) ---")
        # Pandas 출력 옵션 설정 (줄바꿈 등)
        pd.set_option('display.max_colwidth', None) # 전체 내용 표시
        pd.set_option('display.width', 1000) # 너비 확장
        print(qualitative_comparison_df)
        
        # 결과를 CSV 파일로 저장
        qualitative_csv_path = os.path.join(analysis_root_dir, 'qualitative_comparison_samples_batchsize.csv')
        try:
            qualitative_comparison_df.to_csv(qualitative_csv_path, index=False, encoding='utf-8-sig')
            print(f"\n정성적 비교 샘플을 CSV 파일로 저장했습니다: {qualitative_csv_path}")
        except Exception as e:
            print(f"오류: 정성적 비교 샘플 CSV 저장 실패 - {e}")
    else:
        print("정성적 비교 데이터프레임 생성에 실패했습니다.")
else:
    print("평가 결과가 없어 정성적 비교를 수행할 수 없습니다.")

## 10. 결론 및 요약

### 분석 요약

이 분석에서는 원본 HyperCLOVAX 모델과 Batch Size 1, 2, 4로 각각 미세조정된 LoRA 모델의 성능을 비교했습니다.

**주요 평가지표:**
- BLEU, ROUGE (1, 2, L): 번역 및 요약 품질 평가 지표
- 문자 정확도 (Character Accuracy): 생성된 텍스트와 정답 텍스트 간의 문자 수준 유사도
- 정확 일치율 (Exact Match Rate): 생성된 텍스트가 정답과 완전히 일치하는 비율
- 평균 추론 시간: 샘플 당 텍스트 생성에 소요된 평균 시간

**결과 해석:**

*   **[Batch Size별 성능 경향 요약]** 
    (예시) Batch Size가 증가함에 따라 BLEU 및 ROUGE 점수가 [증가/감소/유사]하는 경향을 보였습니다. 
    문자 정확도와 정확 일치율은 Batch Size [값]에서 가장 높게 나타났습니다.
    Batch Size가 성능에 미치는 영향은 [크다/작다/미미하다]고 판단됩니다.

*   **[추론 시간]** 
    (예시) 평균 추론 시간은 모든 모델에서 유사하게 나타났으며, Batch Size 변경이 추론 속도에 큰 영향을 주지 않았습니다. 
    (또는) Batch Size [값] 모델이 가장 [빠른/느린] 추론 속도를 보였습니다.

*   **[정성적 분석]** 
    (예시) 실제 생성된 텍스트 예시를 비교한 결과, Batch Size [값] 모델이 문맥을 더 잘 파악하고 자연스러운 문장을 생성하는 경향이 있었습니다. 
    원본 모델 대비 미세조정된 모델들이 [개선된 점/특이사항]을 보였습니다.

**최종 결론:**

(예시) 종합적으로 고려했을 때, Batch Size [값]으로 미세조정한 모델이 성능과 효율성 측면에서 가장 균형 잡힌 결과를 보여주었습니다. 
특정 Batch Size가 다른 크기에 비해 [우수한 점]을 나타냈으며, 이는 [이유 추론] 때문일 수 있습니다. 
향후 모델 개선 방향으로는 [제안] 등을 고려해볼 수 있습니다.

*(주의: 위 결과 해석 및 결론은 실제 실행 결과에 따라 달라지므로, 노트북 실행 후 해당 섹션을 구체적인 수치와 관찰 내용으로 채워야 합니다.)*